In [ ]:
using ColoringNames


In [ ]:
cldata = load_munroe_data(dev_as_train=false, dev_as_test=true)

In [ ]:
function qualitative_demo(mdl, do_oov=true)
    names = ["brownish green", "brown", "brownish", "greenish", "greenish brown", "green"]
    if do_oov
        append!(names, ["ish"])
    end
    
    cols = hsv2colorant.(query.(noml, names))
    display(cols)
    println(join(rpad.(names, 10), "|"))
end

qualitative_demo(mdl::TermToColorPointEmpirical) = qualitative_demo(mdl, false)

In [5]:
noml = ColoringNames.TermToColorPointEmpirical()
train!(noml, cldata)
@show evaluate(noml, cldata.dev)

LoadError: [91mMethodError: no method matching mse_from_peak(::Array{Float32,2}, ::Array{Float32,2})[0m
Closest candidates are:
  mse_from_peak(::AbstractArray{T,2} where T, [91m::Tuple{T<:(AbstractArray{T,2} where T),T<:(AbstractArray{T,2} where T),T<:(AbstractArray{T,2} where T)}[39m) where T<:(AbstractArray{T,2} where T) at /home/wheel/oxinabox/.julia/v0.6/ColoringNames/src/evaluation.jl:75[39m

In [ ]:
1

In [ ]:
qualitative_demo(noml)

In [ ]:
function hsv_mean(colors)
    hs = @view(colors[:,1])
    s = mean(@view(colors[:,2]))
    v = mean(@view(colors[:,3]))
    
    ch = mean(cos.(2π*hs))
    sh = mean(sin.(2π*hs))
    h = atan2(sh, ch)/2π
    
    (h, s, v)
end

In [ ]:
hsquared_error([0.9, 0.9], [0.7,0.1])